In [57]:
import pandas as pd
import numpy as np


In [58]:
data = pd.read_csv('f1_public.csv')
print(data.head())

   UniID  AInjDt  AAdmDt  ARbAdmDt  ADisDt AInjAge  ARbAdDtM  ASDDAc2R  \
0      1    1982    1982    1982.0    1982  15-29y         1       888   
1      2    1982    1982    1982.0    1982  15-29y         1       888   
2      3    1982    1982    1982.0    1982  15-29y         1       888   
3      4    1982    1982    1982.0    1983  15-29y         1       888   
4      5    1982    1982    1982.0    1982  15-29y         1       888   

   ASDDDurR  AI2ADays  ...  AFTltgDs  AFBdMgDs  AFBwMgDs  AFMBCWDs  AFMTltDs  \
0         0        51  ...         9         9         9         9         9   
1         0        24  ...         9         9         9         9         9   
2         2        13  ...         9         9         9         9         9   
3         0        26  ...         9         9         9         9         9   
4         0        32  ...         9         9         9         9         9   

   AFMTShDs  AFLWWcDs  AFLModDs AFLStrDs  AFScorDs  
0         9         9

<ipython-input-58-64e4ffc99551>:1: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('f1_public.csv')


In [59]:
def recategorizing_ASIA_Scale(row):
    # Convert Frankel Scale to ASIA Scale
    if row['AASAImDs'] == '5':
        return 'A'
    elif row['AASAImDs'] == '4':
        return 'E'
    elif row['AASAImDs'] == '3':
        return 'D'
    elif row['AASAImDs'] == '2':
        return 'C'
    elif row['AASAImDs'] == '1':
        return 'B'
    elif row['AASAImDs'] == '9':
        return 'U'
    else:
        return row['AASAImDs']

data['AASAImDs'] = data.apply(recategorizing_ASIA_Scale, axis=1)


In [60]:
data['AASAImDs'].value_counts()

AASAImDs
A    13891
D     9375
C     3909
B     3448
U     1352
E      184
Name: count, dtype: int64

In [61]:
data.count()

UniID       32159
AInjDt      32159
AAdmDt      32159
ARbAdmDt    31371
ADisDt      32159
            ...  
AFMTShDs    32159
AFLWWcDs    32159
AFLModDs    32159
AFLStrDs    32159
AFScorDs    32158
Length: 417, dtype: int64

In [62]:
# New Feature Creation - Neurologic Level of Injury (NLI_cat)

def categorize_nli(row):
    # Ensure 'ANurLvlD' is treated as a string, safely handling NaN values
    anlvl = str(row['ANurLvlD'])
    # Check the ASIA impairment scale
    if row['AASAImDs'] in ['A', 'B', 'C']:
        # Check the neurological level of injury for cervical
        if any(anlvl.startswith(x) for x in ('C01', 'C02', 'C03', 'C04', 'C05', 'C06', 'C07', 'C08', 'C99')):
            return 'C'
        # Check for thoracic level
        elif any(anlvl.startswith(x) for x in ('T01', 'T02', 'T03', 'T04', 'T05', 'T06', 'T07', 'T08', 'T09', 'T10', 'T11', 'T12', 'T99')):
            return 'T'
        # Check for lumbar level
        elif any(anlvl.startswith(x) for x in ('L01', 'L02', 'L03', 'L04', 'L05', 'L99')):
            return 'L'
        # Check for sacral level
        elif any(anlvl.startswith(x) for x in ('S01', 'S02', 'S03', 'S99')):
            return 'S'
    # Check if ASIA impairment scale is 'D'
    elif row['AASAImDs'] == 'D':
        return 'D'
    # Return NaN for 'E' or 'U' or any other conditions
    return np.nan

# Applying the function to create the 'NLI_cat' column
data['NLI_cat'] = data.apply(categorize_nli, axis=1)

# Remove rows with undefined NLI_cat (NaNs)
data.dropna(subset=['NLI_cat'], inplace=True)


In [63]:
data.head()

,UniID,AInjDt,AAdmDt,ARbAdmDt,ADisDt,AInjAge,ARbAdDtM,ASDDAc2R,ASDDDurR,AI2ADays,...,AFBdMgDs,AFBwMgDs,AFMBCWDs,AFMTltDs,AFMTShDs,AFLWWcDs,AFLModDs,AFLStrDs,AFScorDs,NLI_cat
0,1,1982,1982,1982.0,1982,15-29y,1,888,0,51,...,9,9,9,9,9,9,9,9,99.0,C
1,2,1982,1982,1982.0,1982,15-29y,1,888,0,24,...,9,9,9,9,9,9,9,9,99.0,T
2,3,1982,1982,1982.0,1982,15-29y,1,888,2,13,...,9,9,9,9,9,9,9,9,99.0,D
3,4,1982,1982,1982.0,1983,15-29y,1,888,0,26,...,9,9,9,9,9,9,9,9,99.0,C
4,5,1982,1982,1982.0,1982,15-29y,1,888,0,32,...,9,9,9,9,9,9,9,9,99.0,C


In [64]:
# Preview the first 5 rows of the updated DataFrame
# NLI_cat exists
print(data[['ANurLvlD','NLI_cat', 'AASAImDs']].head())

  ANurLvlD NLI_cat AASAImDs
0      C04       C        A
1      T10       T        A
2      X99       D        D
3      C05       C        A
4      C06       C        A


In [65]:
#data.to_csv('updated_data.csv', index=False)
print(data['ATrmEtio'].count())

30227


In [66]:
# Create Etiology Category Variable
def categorize_etiology(etio):
    if etio in [20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 70, 71, 72, 73, 74, 75, 76, 77, 78]:
        return 'SportEtiol'
    elif etio in [1, 2, 3, 4, 5, 6, 7, 8, 9]:
        return 'VehicleEtiol'
    elif etio in [10, 11, 12, 15, 30, 31, 40, 50, 60, 99]:
        return 'OtherEtiol'
    else:
        return np.nan

data['Etiology_cat'] = data['ATrmEtio'].apply(categorize_etiology)


In [67]:
def categorize_core_injury_severity(row):
    anlvl = str(row['ANurLvlD'])

    # Check if they needed any type of mechanical ventilator post discharge
    if row['AUMVDis'] in ['1', '2', '3', '4', '5']:
        return 'CoreVent'
    # Check the ASIA impairment scale
    # What do we do with 'C99 or L99, or T99'?
    elif row['AASAImDs'] in ['A', 'B', 'C']:
        # Check the neurological level of injury for cervical
        if any(anlvl.startswith(x) for x in ('C01', 'C02', 'C03', 'C04')):
            return 'CoreHighC'
        # Check the neurological level of injury for cervical
        if any(anlvl.startswith(x) for x in ('C05', 'C06', 'C07', 'C08')):
            return 'CoreLowC'
        # Check for thoracic level
        elif any(anlvl.startswith(x) for x in ('T01', 'T02', 'T03', 'T04', 'T05', 'T06', 'T07', 'T08', 'T09', 'T10', 'T11', 'T12', 'L01', 'L02', 'L03', 'L04', 'L05', 'S01', 'S02', 'S03', 'S99')):
            return 'CorePara'
    # Check if ASIA impairment scale is 'D'
    elif row['AASAImDs'] == 'D':
        return 'CoreD'

data['CoreInjurySeverity'] = data.apply(categorize_nli, axis=1)


In [68]:
# IDENTIFY VARIABLES THAT MAY HAVE MISSING VALUES
total_counts = pd.Series(data.count())
total_counts[total_counts != 30227]

ARbAdmDt    29720
AInCatch     7012
AExtCsIj    30226
AVertInj     7027
AAsscInj     7024
            ...  
ANurLvlR    30176
AAnSnRhb     9600
AVoSphRb     9600
AASAImRb    30176
ANCatRhb    30176
Length: 79, dtype: int64

In [69]:
# Check Ventilator dependent

# Check the counts for AUMVAdm
aume_adm_counts = filtered_data['AUMVAdm'].value_counts()
print("Counts for AUMVAdm:")
print(aume_adm_counts)

# Check the counts for AUMVDis
aume_dis_counts = filtered_data['AUMVDis'].value_counts()
print("Counts for AUMVDis:")
print(aume_dis_counts)


Counts for AUMVAdm:
AUMVAdm
0.0    5724
1.0     442
2.0     379
9.0      59
3.0       2
Name: count, dtype: int64
Counts for AUMVDis:
AUMVDis
0    6369
2     124
1      78
9      30
3       5
Name: count, dtype: int64


In [70]:
# Define the function to categorize CoreInjurySeverity
def categorize_core_injury_severity(row):
    anlvl = str(row['ANurLvlD'])
    ais = row['AASAImDs']

    # CoreVent: Ventilator dependent at any NLI or with any AIS grade, at admission or discharge
    if row['AUMVAdm'] in [1, 2, 3, 4, 5] or row['AUMVDis'] in [1, 2, 3, 4, 5]:
        return 'CoreVent'

    # CoreHighC: C1–4 AIS A, B, and C
    if ais in ['A', 'B', 'C'] and any(anlvl.startswith(x) for x in ['C01', 'C02', 'C03', 'C04']):
        return 'CoreHighC'

    # CoreLowC: C5–8 AIS A, B, and C
    if ais in ['A', 'B', 'C'] and any(anlvl.startswith(x) for x in ['C05', 'C06', 'C07', 'C08']):
        return 'CoreLowC'

    # CorePara: T1–S3 AIS A, B, and C
    if ais in ['A', 'B', 'C'] and any(anlvl.startswith(x) for x in ['T01', 'T02', 'T03', 'T04', 'T05', 'T06', 'T07', 'T08', 'T09', 'T10', 'T11', 'T12', 'L01', 'L02', 'L03', 'L04', 'L05', 'S01', 'S02', 'S03']):
        return 'CorePara'

    # CoreD: AIS D at any NLI
    if ais == 'D':
        return 'CoreD'

    return np.nan

# Apply the function to create the 'CoreInjurySeverity' column
data['CoreInjurySeverity'] = data.apply(categorize_core_injury_severity, axis=1)

# Check the distribution of the new variable
print(data['CoreInjurySeverity'].value_counts())


CoreInjurySeverity
CorePara     9843
CoreD        8869
CoreLowC     4898
CoreVent     4327
CoreHighC    2250
Name: count, dtype: int64


## DATA EXPLORATION

In [71]:
# Get Gender Breakdown
data.groupby(['ASex']).size()

ASex
1    24426
2     5799
3        2
dtype: int64

In [72]:
# Get Age Breakdown
data.groupby(['AInjAge']).size()

AInjAge
0-14y       523
15-29y    14531
30-44y     7236
45-59y     4708
60-74y     2508
75+y        721
dtype: int64

In [73]:
# Get Discharged Cohorts
decades = data.groupby(['ADisDt']).size().reset_index()
decades['Decade'] = (decades['ADisDt'] // 10) * 10
decades.groupby('Decade')[0].sum().reset_index()

,Decade,0
0,1970,4008
1,1980,8758
2,1990,6552
3,2000,6434
4,2010,4475


In [74]:
# Get Age Breakdown
data.groupby(['Etiology_cat', 'AInjAge', 'ASex']).size()

Etiology_cat  AInjAge  ASex
OtherEtiol    0-14y    1        165
                       2         61
              15-29y   1       4894
                       2        710
                       3          1
              30-44y   1       2954
                       2        530
              45-59y   1       2214
                       2        468
              60-74y   1       1260
                       2        441
              75+y     1        348
                       2        210
SportEtiol    0-14y    1         76
                       2         34
              15-29y   1       1978
                       2        193
              30-44y   1        481
                       2         59
              45-59y   1        186
                       2         31
              60-74y   1         65
                       2          9
              75+y     1          6
VehicleEtiol  0-14y    1        108
                       2         79
              15-29y   1       5203


In [75]:
# Analyze Stature Data

# 999 is a placeholder and should be excluded
filtered_data = data[data['AHghtRhb'] != 999]

In [76]:
filtered_data.groupby(['Etiology_cat', 'AInjAge', 'ASex']).size()

Etiology_cat  AInjAge  ASex
OtherEtiol    0-14y    1         7
                       2         2
              15-29y   1       814
                       2       116
                       3         1
              30-44y   1       552
                       2        90
              45-59y   1       734
                       2       186
              60-74y   1       509
                       2       189
              75+y     1       144
                       2        81
SportEtiol    0-14y    1         5
                       2         3
              15-29y   1       275
                       2        28
              30-44y   1       107
                       2        16
              45-59y   1        92
                       2        13
              60-74y   1        37
                       2         4
              75+y     1         3
VehicleEtiol  0-14y    1         8
                       2         4
              15-29y   1       785
                       2   

In [77]:
# Apply the function to create the 'CoreInjurySeverity' column
filtered_data['CoreInjurySeverity'] = filtered_data.apply(categorize_core_injury_severity, axis=1)

# Check the distribution of the new variable
print(filtered_data['CoreInjurySeverity'].value_counts())

# Preview the first 5 rows of the updated DataFrame
print(filtered_data[['ANurLvlD', 'AASAImDs', 'AUMVDis', 'CoreInjurySeverity']].head())

CoreInjurySeverity
CoreD        2364
CorePara     2000
CoreVent      833
CoreLowC      715
CoreHighC     675
Name: count, dtype: int64
     ANurLvlD AASAImDs  AUMVDis CoreInjurySeverity
1821      T10        D        0              CoreD
1822      C05        A        0           CoreLowC
1823      T10        B        0           CorePara
1824      T10        A        0           CorePara
1825      C05        A        0           CoreLowC


<ipython-input-77-57ec05baea17>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['CoreInjurySeverity'] = filtered_data.apply(categorize_core_injury_severity, axis=1)


In [78]:
# Recalculate descriptive statistics
height_descriptives = filtered_data['AHghtRhb'].describe()
print(height_descriptives)

# Calculate quintiles with cleaned data
filtered_data['QuinHgt'] = pd.qcut(filtered_data['AHghtRhb'], 5, labels=[1, 2, 3, 4, 5], duplicates='drop')

# Check new quintile assignments
print(filtered_data['QuinHgt'].value_counts())

# Group by 'NLI_cat' and analyze height
grouped_nli = filtered_data.groupby('NLI_cat')['AHghtRhb'].agg(['mean', 'median', 'std'])
print(grouped_nli)



count    6606.000000
mean       69.132001
std         3.960152
min        34.000000
25%        67.000000
50%        70.000000
75%        72.000000
max        83.000000
Name: AHghtRhb, dtype: float64
QuinHgt
1    1615
4    1403
3    1316
5    1228
2    1044
Name: count, dtype: int64
              mean  median       std
NLI_cat                             
C        69.311080    70.0  4.102351
D        69.014475    70.0  3.909109
L        68.817073    69.0  3.637649
S        68.500000    68.5  7.778175
T        69.138030    69.0  3.904427


<ipython-input-78-17054c7c5f82>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['QuinHgt'] = pd.qcut(filtered_data['AHghtRhb'], 5, labels=[1, 2, 3, 4, 5], duplicates='drop')


In [79]:
grouped_nli = filtered_data.groupby('NLI_cat')['AHghtRhb'].agg(['mean', 'median', 'std'])


In [80]:
# Group by 'Etiology_cat' and gender
grouped_nli = filtered_data.groupby(['Etiology_cat', 'ASex'])['AHghtRhb'].agg(['mean', 'median', 'std'])
print(grouped_nli)

                        mean  median       std
Etiology_cat ASex                             
OtherEtiol   1     70.074638    70.0  3.253494
             2     64.421687    64.0  3.137841
             3     69.000000    69.0       NaN
SportEtiol   1     70.828516    71.0  3.038917
             2     64.437500    64.0  3.245877
VehicleEtiol 1     70.432716    71.0  3.217251
             2     64.700669    65.0  3.297992
             3     65.000000    65.0       NaN


In [81]:
# Percentile Height Check
len(filtered_data[filtered_data['AHghtRhb'] >= 73]) / len(filtered_data)
len(filtered_data[filtered_data['AHghtRhb'] >= 72]) / len(filtered_data)

len(filtered_data[(filtered_data['AHghtRhb'] >= 75) & (filtered_data['ASex'] == 2)]) / len(filtered_data)
len(filtered_data[(filtered_data['AHghtRhb'] >= 69) & (filtered_data['ASex'] == 2)]) / len(filtered_data)

len(filtered_data[(filtered_data['AHghtRhb'] >= 74) & (filtered_data['ASex'] == 1)]) / len(filtered_data)
len(filtered_data[(filtered_data['AHghtRhb'] >= 68) & (filtered_data['ASex'] == 2)]) / len(filtered_data)


0.029064486830154404

In [82]:
# Calculate Percentiles
percentiles = {
    95: np.percentile(filtered_data['AHghtRhb'], 95),
    90: np.percentile(filtered_data['AHghtRhb'], 90)
}

# Filter data based on these thresholds
above_95 = filtered_data[filtered_data['AHghtRhb'] >= percentiles[95]]
above_90 = filtered_data[filtered_data['AHghtRhb'] >= percentiles[90]]

np.percentile(filtered_data[filtered_data['ASex'] == 2]['AHghtRhb'], 95)

69.0

In [83]:
# Detailed Analysis by Etiology Category
sports_data = filtered_data[filtered_data['Etiology_cat'] == 'SportEtiol']
vehicular_data = filtered_data[filtered_data['Etiology_cat'] == 'VehicleEtiol']
others_data = filtered_data[filtered_data['Etiology_cat'] == 'OtherEtiol']

# Calculate descriptive statistics for each group
sports_descriptives = sports_data.groupby('NLI_cat')['AHghtRhb'].agg(['mean', 'median', 'std'])
vehicular_descriptives = vehicular_data.groupby('NLI_cat')['AHghtRhb'].agg(['mean', 'median', 'std'])
other_descriptives = others_data.groupby('NLI_cat')['AHghtRhb'].agg(['mean', 'median', 'std'])

print(sports_descriptives)
print(vehicular_descriptives)
print(other_descriptives)



              mean  median       std
NLI_cat                             
C        70.364516    71.0  3.504879
D        70.189055    71.0  3.746209
L        69.066667    69.0  3.104528
S        74.000000    74.0       NaN
T        68.803571    69.0  4.020039
              mean  median       std
NLI_cat                             
C        69.111957    70.0  4.201853
D        69.283493    70.0  3.881449
L        68.333333    68.0  3.892413
S        63.000000    63.0       NaN
T        69.025503    69.0  4.006801
              mean  median       std
NLI_cat                             
C        69.148526    70.0  4.140213
D        68.680666    69.0  3.906161
L        69.013825    69.0  3.549230
T        69.246561    69.0  3.815328


In [84]:
# Calculate the distribution of CoreInjurySeverity by height quintiles
core_injury_quintile_distribution = filtered_data.groupby(['CoreInjurySeverity', 'QuinHgt']).size().unstack()
print(core_injury_quintile_distribution)

# Calculate percentages for specified heights
percent_73_or_greater = len(filtered_data[filtered_data['AHghtRhb'] >= 73]) / len(filtered_data)
percent_men_75_or_greater = len(filtered_data[(filtered_data['AHghtRhb'] >= 75) & (filtered_data['ASex'] == 1)]) / len(filtered_data[filtered_data['ASex'] == 1])
percent_women_69_or_greater = len(filtered_data[(filtered_data['AHghtRhb'] >= 69) & (filtered_data['ASex'] == 2)]) / len(filtered_data[filtered_data['ASex'] == 2])

# Print results
print("Percentage of persons with SCI who are 73” and greater: {:.2%}".format(percent_73_or_greater))
print("Percentage of men with SCI who are 75” or greater: {:.2%}".format(percent_men_75_or_greater))
print("Percentage of women with SCI who are 69” or greater: {:.2%}".format(percent_women_69_or_greater))

# Calculate percentages for specified heights
percent_72_or_greater = len(filtered_data[filtered_data['AHghtRhb'] >= 72]) / len(filtered_data)
percent_men_74_or_greater = len(filtered_data[(filtered_data['AHghtRhb'] >= 74) & (filtered_data['ASex'] == 1)]) / len(filtered_data[filtered_data['ASex'] == 1])
percent_women_68_or_greater = len(filtered_data[(filtered_data['AHghtRhb'] >= 68) & (filtered_data['ASex'] == 2)]) / len(filtered_data[filtered_data['ASex'] == 2])

# Print results
print("Percentage of persons with SCI who are 72” and greater: {:.2%}".format(percent_72_or_greater))
print("Percentage of men with SCI who are 74” or greater: {:.2%}".format(percent_men_74_or_greater))
print("Percentage of women with SCI who are 68” or greater: {:.2%}".format(percent_women_68_or_greater))

# Calculate percentiles within etiology categories
def calculate_percentiles(group):
    return {
        '90th_percentile': np.percentile(group, 90),
        '95th_percentile': np.percentile(group, 95)
    }

percentiles_within_etiology = filtered_data.groupby('Etiology_cat')['AHghtRhb'].apply(calculate_percentiles).unstack()
print(percentiles_within_etiology)

# Calculate descriptive statistics
height_descriptives = filtered_data['AHghtRhb'].describe()
print(height_descriptives)

# Calculate quintiles and assign each person a quintile
filtered_data['QuinHgt'] = pd.qcut(filtered_data['AHghtRhb'], 5, labels=[1, 2, 3, 4, 5], duplicates='drop')

# Verify the quintile assignments
print(filtered_data['QuinHgt'].value_counts())

# Group by NLI category and height quintile, then calculate descriptive statistics
nli_descriptive_stats = filtered_data.groupby(['NLI_cat', 'QuinHgt'])['AHghtRhb'].agg(['mean', 'median', 'std'])
print(nli_descriptive_stats)

# Group by CoreInjurySeverity category and height quintile, then calculate descriptive statistics
core_injury_descriptive_stats = filtered_data.groupby(['CoreInjurySeverity', 'QuinHgt'])['AHghtRhb'].agg(['mean', 'median', 'std'])
print(core_injury_descriptive_stats)

# For height, calculate mean, median, and standard deviation for each NLI using the two etiologies
sports_data = filtered_data[filtered_data['Etiology_cat'] == 'SportEtiol']
vehicular_data = filtered_data[filtered_data['Etiology_cat'] == 'VehicleEtiol']

sports_descriptive_stats = sports_data.groupby('NLI_cat')['AHghtRhb'].agg(['mean', 'median', 'std'])
vehicular_descriptive_stats = vehicular_data.groupby('NLI_cat')['AHghtRhb'].agg(['mean', 'median', 'std'])

print("Sports Etiology NLI Descriptives:\n", sports_descriptive_stats)
print("Vehicle Etiology NLI Descriptives:\n", vehicular_descriptive_stats)


QuinHgt               1    2    3    4    5
CoreInjurySeverity                         
CoreD               616  363  464  516  405
CoreHighC           149  118  121  161  126
CoreLowC            169  103  155  143  145
CorePara            485  341  419  405  350
CoreVent            192  115  156  177  193
Percentage of persons with SCI who are 73” and greater: 18.59%
Percentage of men with SCI who are 75” or greater: 8.01%
Percentage of women with SCI who are 69” or greater: 8.75%
Percentage of persons with SCI who are 72” and greater: 30.76%
Percentage of men with SCI who are 74” or greater: 15.04%
Percentage of women with SCI who are 68” or greater: 14.48%
              90th_percentile  95th_percentile
Etiology_cat                                  
OtherEtiol               74.0             75.0
SportEtiol               74.0             76.0
VehicleEtiol             74.0             75.0
count    6606.000000
mean       69.132001
std         3.960152
min        34.000000
25%        67

<ipython-input-84-8997097dfb4a>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['QuinHgt'] = pd.qcut(filtered_data['AHghtRhb'], 5, labels=[1, 2, 3, 4, 5], duplicates='drop')


In [85]:
# Odds Ratio Analysis
from scipy.stats import chi2_contingency

# Function to calculate odds ratio
def calculate_odds_ratio(data, condition):
    contingency_table = pd.crosstab(condition, data['AHghtRhb'] >= 73)
    chi2, p, _, _ = chi2_contingency(contingency_table)
    odds_ratio = (contingency_table.iloc[1, 1] / contingency_table.iloc[1, 0]) / (contingency_table.iloc[0, 1] / contingency_table.iloc[0, 0])
    return odds_ratio, p

# Calculate odds ratios for each CoreInjurySeverity category
categories = ['CoreHighC', 'CoreLowC', 'CorePara', 'CoreD', 'CoreVent']

results = {}
for category in categories:
    condition = filtered_data['CoreInjurySeverity'] == category
    if condition.sum() > 0:  # Ensure there is data for the category
        odds_ratio, p_value = calculate_odds_ratio(filtered_data, condition)
        results[category] = (odds_ratio, p_value)

for category, (odds_ratio, p_value) in results.items():
    print(f"Odds Ratio for {category}: {odds_ratio} P-value: {p_value}")


Odds Ratio for CoreHighC: 1.0057124155782333 P-value: 0.9980703660884813
Odds Ratio for CoreLowC: 1.1293515413649544 P-value: 0.2381533112622589
Odds Ratio for CorePara: 0.9006695658176296 P-value: 0.1429014375556241
Odds Ratio for CoreD: 0.858855008847845 P-value: 0.025106832564657396
Odds Ratio for CoreVent: 1.3804861111111113 P-value: 0.0003341342080771632


In [86]:
# Dataframes
nli_quintile_distribution = filtered_data.groupby(['NLI_cat', 'QuinHgt']).size().unstack()
etiology_quintile_distribution = filtered_data.groupby(['Etiology_cat', 'QuinHgt']).size().unstack()
height_descriptives = filtered_data['AHghtRhb'].describe()
nli_descriptive_stats = filtered_data.groupby(['NLI_cat', 'QuinHgt'])['AHghtRhb'].agg(['mean', 'median', 'std'])
core_injury_descriptive_stats = filtered_data.groupby(['CoreInjurySeverity', 'QuinHgt'])['AHghtRhb'].agg(['mean', 'median', 'std'])

# Export to Excel
with pd.ExcelWriter('results.xlsx') as writer:
    nli_quintile_distribution.to_excel(writer, sheet_name='NLI_Quintile_Distribution')
    etiology_quintile_distribution.to_excel(writer, sheet_name='Etiology_Quintile_Distribution')
    height_descriptives.to_frame().to_excel(writer, sheet_name='Height_Descriptives')  # Convert Series to DataFrame
    nli_descriptive_stats.to_excel(writer, sheet_name='NLI_Descriptive_Stats')
    core_injury_descriptive_stats.to_excel(writer, sheet_name='Core_Injury_Descriptive_Stats')
